In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
# First we go one back in our directory to access the folder with our functions
%cd ..

# Now we import the functions from the features folder
# This is the functions we have created to generate features for electricity prices, weather measures, and the danish calendar
from features import electricity_prices, weather_measures, calendar

# We go back into the notebooks folder
%cd pipeline

/Users/tobiasmjensen/Documents/aau_bds/m7_second-semester-project/bds_m7_second-semester-project
/Users/tobiasmjensen/Documents/aau_bds/m7_second-semester-project/bds_m7_second-semester-project/pipeline


In [3]:
# Fetching historical electricity prices for area DK1 from January 1, 2022
# Note: The end date is currently left out to retrieve data up to the day before present date 
# Today is not included in the data as it is not historical data
electricity_df = electricity_prices.electricity_prices(
    historical=True, 
    area=["DK1"], 
    start='2022-01-01'
)

# electricity_df

In [4]:
# Check if there are any missing values in the dataframe
electricity_df[electricity_df['dk1_spotpricedkk_kwh'].isna()]

,timestamp,datetime,date,hour,dk1_spotpricedkk_kwh


In [5]:
# Defining start and end year for the calendar data
start='2022-01-01'
end=datetime.now().date() + timedelta(days=7)

print(f"Fetching historical calendar from {start} to {end}")

Fetching historical calendar from 2022-01-01 to 2024-05-30


In [6]:
# Fetching the Danish calendar from January 1, 2022 to December 31, 2025.
# This is dynamic and will update the calendar to the current year plus one year.
calendar_df = calendar.calendar_denmark(
    freq='H',
    start=start, 
    end=end
)

# calendar_df

In [7]:
# Merging the weather forecast and calendar dataframes
new_data = pd.merge(electricity_df, calendar_df, how='right', left_on='timestamp', right_on='timestamp')

# Drop columns 'date', 'datetime', and 'timestamp' from the DataFrame 'new_data' to match the training data schema
data = new_data.drop(columns=['date_x', 'datetime_x', 'hour_x'])
data = data.rename(columns={'date_y': 'date', 'datetime_y': 'datetime', 'hour_y': 'hour'})

# Displaying the last 5 rows of the new data
data

,timestamp,dk1_spotpricedkk_kwh,datetime,date,hour,dayofweek,day,month,year,workday
0,1640995200000,0.37220,2022-01-01 00:00:00,2022-01-01,0,5,1,1,2022,0
1,1640998800000,0.30735,2022-01-01 01:00:00,2022-01-01,1,5,1,1,2022,0
2,1641002400000,0.32141,2022-01-01 02:00:00,2022-01-01,2,5,1,1,2022,0
3,1641006000000,0.33806,2022-01-01 03:00:00,2022-01-01,3,5,1,1,2022,0
4,1641009600000,0.28013,2022-01-01 04:00:00,2022-01-01,4,5,1,1,2022,0
...,...,...,...,...,...,...,...,...,...,...
21139,1717095600000,NaN,2024-05-30 19:00:00,2024-05-30,19,3,30,5,2024,1
21140,1717099200000,NaN,2024-05-30 20:00:00,2024-05-30,20,3,30,5,2024,1
21141,1717102800000,NaN,2024-05-30 21:00:00,2024-05-30,21,3,30,5,2024,1
21142,1717106400000,NaN,2024-05-30 22:00:00,2024-05-30,22,3,30,5,2024,1


In [8]:
data[data['dk1_spotpricedkk_kwh'].isna()]

,timestamp,dk1_spotpricedkk_kwh,datetime,date,hour,dayofweek,day,month,year,workday
2042,1648346400000,NaN,2022-03-27 02:00:00,2022-03-27,2,6,27,3,2022,0
10778,1679796000000,NaN,2023-03-26 02:00:00,2023-03-26,2,6,26,3,2023,0
19682,1711850400000,NaN,2024-03-31 02:00:00,2024-03-31,2,6,31,3,2024,0
20952,1716422400000,NaN,2024-05-23 00:00:00,2024-05-23,0,3,23,5,2024,1
20953,1716426000000,NaN,2024-05-23 01:00:00,2024-05-23,1,3,23,5,2024,1
...,...,...,...,...,...,...,...,...,...,...
21139,1717095600000,NaN,2024-05-30 19:00:00,2024-05-30,19,3,30,5,2024,1
21140,1717099200000,NaN,2024-05-30 20:00:00,2024-05-30,20,3,30,5,2024,1
21141,1717102800000,NaN,2024-05-30 21:00:00,2024-05-30,21,3,30,5,2024,1
21142,1717106400000,NaN,2024-05-30 22:00:00,2024-05-30,22,3,30,5,2024,1


In [9]:
data2 = data.copy()

# Fill NaN values with value from previous row if date is before today
today = datetime.today().date()
mask = data2['datetime'].dt.date < today
data2.loc[mask, 'dk1_spotpricedkk_kwh'] = data2['dk1_spotpricedkk_kwh'].ffill()

data2[data2['dk1_spotpricedkk_kwh'].isna()]

,timestamp,dk1_spotpricedkk_kwh,datetime,date,hour,dayofweek,day,month,year,workday
20952,1716422400000,NaN,2024-05-23 00:00:00,2024-05-23,0,3,23,5,2024,1
20953,1716426000000,NaN,2024-05-23 01:00:00,2024-05-23,1,3,23,5,2024,1
20954,1716429600000,NaN,2024-05-23 02:00:00,2024-05-23,2,3,23,5,2024,1
20955,1716433200000,NaN,2024-05-23 03:00:00,2024-05-23,3,3,23,5,2024,1
20956,1716436800000,NaN,2024-05-23 04:00:00,2024-05-23,4,3,23,5,2024,1
...,...,...,...,...,...,...,...,...,...,...
21139,1717095600000,NaN,2024-05-30 19:00:00,2024-05-30,19,3,30,5,2024,1
21140,1717099200000,NaN,2024-05-30 20:00:00,2024-05-30,20,3,30,5,2024,1
21141,1717102800000,NaN,2024-05-30 21:00:00,2024-05-30,21,3,30,5,2024,1
21142,1717106400000,NaN,2024-05-30 22:00:00,2024-05-30,22,3,30,5,2024,1


In [10]:
# pd.set_option('display.max_rows', 1000)
# data2[(data2['year'] == 2024) & (data2['month'] == 5)]

In [14]:
# Defining a copy of the combined data to avoid modifying the original dataframe
rolled_data2 = data2.copy()

# Adding a column with the mean for the previous 1 week
rolled_data2['prev_1w_mean'] = rolled_data2['dk1_spotpricedkk_kwh'].rolling(window=24*7, min_periods=1).mean()

# Adding a column with the mean for the previous 2 weeks
rolled_data2['prev_2w_mean'] = rolled_data2['dk1_spotpricedkk_kwh'].rolling(window=24*14, min_periods=1).mean()

# Adding a column with the mean for the previous 4 weeks
rolled_data2['prev_4w_mean'] = rolled_data2['dk1_spotpricedkk_kwh'].rolling(window=24*28, min_periods=1).mean()

# Display the last 120 rows of the DataFrame
rolled_data2.tail(120)

,timestamp,dk1_spotpricedkk_kwh,datetime,date,hour,dayofweek,day,month,year,workday,prev_1w_mean,prev_2w_mean,prev_4w_mean
21024,1716681600000,NaN,2024-05-26 00:00:00,2024-05-26,0,6,26,5,2024,0,0.401972,0.269212,0.367549
21025,1716685200000,NaN,2024-05-26 01:00:00,2024-05-26,1,6,26,5,2024,0,0.399179,0.268777,0.367588
21026,1716688800000,NaN,2024-05-26 02:00:00,2024-05-26,2,6,26,5,2024,0,0.396359,0.268586,0.367709
21027,1716692400000,NaN,2024-05-26 03:00:00,2024-05-26,3,6,26,5,2024,0,0.393618,0.268500,0.367932
21028,1716696000000,NaN,2024-05-26 04:00:00,2024-05-26,4,6,26,5,2024,0,0.390882,0.268431,0.368204
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21139,1717095600000,NaN,2024-05-30 19:00:00,2024-05-30,19,3,30,5,2024,1,NaN,0.348668,0.383060
21140,1717099200000,NaN,2024-05-30 20:00:00,2024-05-30,20,3,30,5,2024,1,NaN,0.349709,0.383205
21141,1717102800000,NaN,2024-05-30 21:00:00,2024-05-30,21,3,30,5,2024,1,NaN,0.350812,0.383351
21142,1717106400000,NaN,2024-05-30 22:00:00,2024-05-30,22,3,30,5,2024,1,NaN,0.352078,0.383561


In [12]:
# Defining a copy of the dataframe to avoid modifying the original dataframe
rolled_data = data2.copy()

# Adding a column with the mean for the previous 1 week
rolled_data['prev_1w_mean'] = rolled_data['dk1_spotpricedkk_kwh'].rolling(window=24*7).mean()

# Adding a column with the mean for the previous 2 weeks
rolled_data['prev_2w_mean'] = rolled_data['dk1_spotpricedkk_kwh'].rolling(window=24*14).mean()

# Adding a column with the mean for the previous 4 weeks
rolled_data['prev_4w_mean'] = rolled_data['dk1_spotpricedkk_kwh'].rolling(window=24*28).mean()

# Display the last 5 rows of the rolled data dataframe
rolled_data

,timestamp,dk1_spotpricedkk_kwh,datetime,date,hour,dayofweek,day,month,year,workday,prev_1w_mean,prev_2w_mean,prev_4w_mean
0,1640995200000,0.37220,2022-01-01 00:00:00,2022-01-01,0,5,1,1,2022,0,NaN,NaN,NaN
1,1640998800000,0.30735,2022-01-01 01:00:00,2022-01-01,1,5,1,1,2022,0,NaN,NaN,NaN
2,1641002400000,0.32141,2022-01-01 02:00:00,2022-01-01,2,5,1,1,2022,0,NaN,NaN,NaN
3,1641006000000,0.33806,2022-01-01 03:00:00,2022-01-01,3,5,1,1,2022,0,NaN,NaN,NaN
4,1641009600000,0.28013,2022-01-01 04:00:00,2022-01-01,4,5,1,1,2022,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21139,1717095600000,NaN,2024-05-30 19:00:00,2024-05-30,19,3,30,5,2024,1,NaN,NaN,NaN
21140,1717099200000,NaN,2024-05-30 20:00:00,2024-05-30,20,3,30,5,2024,1,NaN,NaN,NaN
21141,1717102800000,NaN,2024-05-30 21:00:00,2024-05-30,21,3,30,5,2024,1,NaN,NaN,NaN
21142,1717106400000,NaN,2024-05-30 22:00:00,2024-05-30,22,3,30,5,2024,1,NaN,NaN,NaN
